<a href="https://colab.research.google.com/github/MUMADE-TADM/s1_python-gercevas/blob/main/ejercicioknn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Means
Dado un conjunto $X=\{e_1,\dots,e_m \}$

Encontrar una K-partición o conjunto de centroides $Centroides=\{ \mu_1,\ldots,\mu_k \}$

Tal que hace mínimo $\sum^m_{i=0}min_{\mu_j \in Centroides} (|| e_i -\mu_j||^2)$

## Algoritmo Paso a Paso

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import random 

In [ ]:
plt.figure(figsize=(12, 12))

Generamos datos 1500 sintéticos con 3 distribuciones gausianas aleatorias y los mostramos

In [ ]:
n_samples = 1500
random_state = 170
x, y = make_blobs(n_samples=n_samples, random_state=random_state)
X=pd.DataFrame(x)
plt.scatter(X.loc[:, 0], X.loc[:, 1], c=y)
plt.title("Datos Iniciales")

Los elementos de numeros aleatorias se fundamentan en una semilla. Si quiero mantenerlo, tengo que guardar la semilla.

La x son todas las filas de la columna 0.
La y son todas las columnas de la fila 0.
Con make_blob -> me devuelve el x y la Y para pintarlos.

###Inicialización

Generamos los primeros **Centroides** aleatorios

In [ ]:
K=3
Centroides=np.random.random((K,2))*20.0-10.0
plt.scatter(X.loc[:, 0], X.loc[:, 1])
plt.scatter(Centroides[:,0], Centroides[:,1], marker='x',c=[0,1,2])
plt.title("Primeros Centroides")
print(Centroides)

Tres datos aleatorios de dos dimensiones que este comprendidos entre 5.0 y 7.5.

El primer random es el nombre de la libreria y el segundo es la función random. 

Definimos la métrica $||x-\mu||^2$ entre el centroide y los puntos para seleccionar el centroide más cercano para cada punto
como la distancia euclídea entre el punto en dos dimensiones $x=(x_0,x_1)$ y el centroide $\mu=(\mu_0.\mu_1)$

$d(x,\mu)=\sqrt{(x_0 - \mu_0)^2+(x_1 - \mu_1)^2}$

In [ ]:
def distancia(X1,C):
  # C es el vector de Centroides
  #X1 es el punto
  # Construimos el vector de distancias del punto a cada centroide
  arr=[(sum((X1 - c)**2))**0.5 for c in C]
  #seleccionamos el índice (identificador del centroide) de mínimma distancia y los devolvemos
  return np.where(arr == np.amin(arr))[0][0]

Vamos a definir una función distancia, que le vamos a dar un punto y un centroide. Vamos a contruir un arr para cada elemento de C. A ese elemento de C le resto el valor del punto (x1), lo sumo al cuadrado y lo eleveo al cruadrado. Distancia de cada punto a cada uno de los centroides. Me interesa la menor distancia. 
Me devuelve la posición del arr que es la posición mínima. Buscamos el centroide que esta más cerca de ese punto. 

Para todos los puntos calculamos cual es su centoide más cercano

In [ ]:
X['C']=X.apply(distancia,axis=1,C=Centroides)

Apply -> hace -> aplica a cada fila del data frame la función que indicó. En este caso, la función distancia. Tenía dos parámetros, conjunto de centroides. La función apply, la voy a aplicar a cada uno de los datos. Los centroides que necesita la distancia. Quiero aplicar esta distancia por fila

In [ ]:
X['C'].hist()

### Recálculo de los nuevos centroides

In [ ]:
C=X.groupby(['C']).mean().values
if(len(C)<len(Centroides)):
  C=np.vstack([C,np.random.random((len(Centroides)-len(C),2))*20.0-10.0])
plt.scatter(X.loc[:, 0], X.loc[:, 1],c=X['C'])
plt.scatter(C[:,0], C[:,1], marker='x',c=[0,1,2])
plt.title("Nueva asignación")
print(C)

Cojo el data frame, lo agrupo por el valor del centro, obtengo la media y tengo el nuevo centroide. La longitud de C, el número de grupos que tengo. Si la longitud de C, es mayor al grupo que tengo es porque me falta alguno.

salen tres centroides. El if, es para que salgan tres centroides.

### **(A)** Reaguste de los datos con los nuevos centroides

In [ ]:
X['C']=X.iloc[:,:-1].apply(distancia,axis=1,C=C)

recalcular los centroides.

### **(B)** Recáculo de los nuevos centroides

In [ ]:
C1=X.groupby(['C']).mean().values
if(len(C1)<len(Centroides)):
  C1=np.vstack([C1,np.random.random((len(Centroides)-len(C1),2))*20.0-10.0])


In [ ]:
plt.scatter(X.loc[:, 0], X.loc[:, 1],c=X['C'])
plt.scatter(C1[:,0], C1[:,1], marker='x',c=[0,1,2])
plt.title("Nueva asignación")
print(C1)

In [ ]:
diferencia=np.amax(abs(C-C1))
print(C)
C=C1
print(diferencia)
print(C)

Se observa la diferencia entre las dos etapas, previa y actual. Lo que hay en medio es la diferencia máximo entre los dos. 
Los intervalos, son la diferencia entre elemento y elemento. 
la diferencia máxima que hay es de 2.11

Si la diferencia entre los nuevos centrodes C1 y los antíguos no es la adecuada volver a **(A)**

# Sciki-Learn [K-Mean](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)

Esta libreria es la base de todo el machine learning que hay en Python.

Datos -> aprednizaje y test. El uno para construir mi modelo y el otro para controloar (train y test).

Primero-> crear el modelo
Segundo -> ajuste, algo como fit
Predecir con el modelo (explotación del modelo)

¿Qué hace K mean ++ ->? Hace una inspección de datos para ecnotnrar los que tienen una distribución buena de datos. Buscar centroides que se separen en tre ellos de forma uniforme y cubran el máximo de datos. 

Imporatante el número máximo de interacciones

## Ejemplo

In [ ]:
# Author: Phil Roth <mr.phil.roth@gmail.com>
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

plt.figure(figsize=(12, 12))

n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

In [ ]:
KM= KMeans(n_clusters=3, random_state=random_state)
y_pred=KM.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_pred)
plt.title("Resultados")

fase de ajuste-> el fin del modelo. 
KM modelo generado (k-means)
Km.fit_predit(x) ajuste del modelo
No necesitamos dividir en test and train porque ya lo estamos haciendo con fit. 

**Atributos** del Objeto K-means

|Attributo|Significado|
|---------|-----------|
|**cluster_centers**| Coordenadas de los centroides|
|**labels**|Grupo de cada punto|
|**inertia**| Suma de los cuadrados de los puntos de cada grupo a su centroide|
|**n_iter**| Número de iteraciones|


Atributos básicos-> el número de cluster de K.
Print: los centros, el prototipo.

In [ ]:
print(KM.cluster_centers_)

## Pasaje del Titatic

In [ ]:
%cd titanic.csv

Para subir archivos que estan desde el ordenador

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))

Leer los datos del pasaje del fichero titanic.csv mediante **pd.load_csv()**

In [ ]:
titanic=pd.read_csv('titanic.csv')
cluster_data = titanic[['Fare','Age']].copy(deep=True)
cluster_data.dropna(axis=0, inplace=True)
cluster_data.sort_values(by=['Fare','Age'], inplace=True)
cluster_array = np.array(cluster_data)
print(cluster_data.head())
plt.scatter(cluster_array[:,0],cluster_array[:,1])

La segunda linea selecciona el año, y lo que le costo el pasaje. Las columnas con las que vamos a trabajar.

El cluster data solamente para trabajar con un data frame que tenga esa columna de datos.

drop.na eliminar todos los datos que tengan ... las caracteristicas especificadas.

Una vez que tenemos los datos sin datos nulos, vamos a ordearnos por los dos campos. Por el precio del pasaje y por el año. En la primera fila, los que tienen menor costo y así.

Cluster array, lo que me costó el pasaje y la edad que tienen. La abcisa es la edad, y la cordenada es lo que costo el pasaje.

In [ ]:
KM=KMeans(n_clusters=2)
y_pred=KM.fit_predict(cluster_array)
plt.scatter(cluster_array[:,0],cluster_array[:,1],c=y_pred)
print(KM.cluster_centers_)

Este es un clúster de dos. Identifica dos tipos de personas, la caracteristica que me da la agrupación es el precio del pasaje, el resto no me interesa. 
¿Qué ha marcado el cluster? La magnitud más grande. No tiene sentido por la diferencia de magnitud.


## Normalizar los datos

Entre los diferentes tipos de normalización de los valores de los datos que podemos hacer están:

- Escalado frente al máximo : $x'=\frac{x}{max(|X|)}$
- Normalización entre el mínimo y máximo: $x'=\frac{x-x_{min}}{x_{max}-x_{min}}$
- Estandarización. Siendo $\mu_x$, $\sigma_x$  la media de y desviación estandar de  X: $x_{std}=\frac{x-\mu}{\sigma}$

Escala entre 0 y 1. Seguridad de que no tengo valores superiores a 1.

normalización estándar -> cualquiera de las magnitudes va a tener media 0 y desviación 1. Por lo cual, se podrá cosntruir modelos con sentido.

### **Escalado con el máximo**

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
# creamos el objeto an abs_scaler
abs_scaler = MaxAbsScaler()
abs_scaler.fit(cluster_array)

paquete preprocessing

In [ ]:
# the maximum absolute values calculated by the fit method
abs_scaler.max_abs_

In [ ]:
#Escalamos los datos
scaled_data = abs_scaler.transform(cluster_array)
plt.scatter(scaled_data[:,0],scaled_data[:,1])

In [ ]:
KM=KMeans(n_clusters=2)
y_pred=KM.fit_predict(scaled_data)
plt.scatter(scaled_data[:,0],scaled_data[:,1],c=y_pred)
print(KM.cluster_centers_)

Ahora nos divide por la variable más relevante que es la edad. Por lo que más agrupa, a diferencia del anterior. De agrupar por costo del billete, a aagrupar por la edad. Por un cambio, de escala.

In [ ]:
abs_scaler.inverse_transform(KM.cluster_centers_)

### Mínimo-Maximo

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# creamos el objeto an abs_scaler
scaler = MinMaxScaler()
scaler.fit(cluster_array)
#Escalamos los datos
scaled_data = abs_scaler.transform(cluster_array)
plt.scatter(scaled_data[:,0],scaled_data[:,1])

In [ ]:
KM=KMeans(n_clusters=2)
y_pred=KM.fit_predict(scaled_data)
plt.scatter(scaled_data[:,0],scaled_data[:,1],c=y_pred)
print(KM.cluster_centers_)

In [ ]:
scaler.inverse_transform(KM.cluster_centers_)

### Estandarizar

In [ ]:
from sklearn.preprocessing import StandardScaler
# creamos el objeto an abs_scaler
standar = StandardScaler()
standar.fit(cluster_array)
#Escalamos los datos
scaled_data = standar.transform(cluster_array)
plt.scatter(scaled_data[:,0],scaled_data[:,1])

Ya no va entre 0 y 1. La media de X es alrededor de 0 y la media de Y es 0. La desviación típica en ambos ejes es de 1.


In [ ]:
KM=KMeans(n_clusters=2)
y_pred=KM.fit_predict(scaled_data)
plt.scatter(scaled_data[:,0],scaled_data[:,1],c=y_pred)
print(KM.cluster_centers_)

In [ ]:
scaler.inverse_transform(KM.cluster_centers_)

## Flores Iris

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
X, y = datasets.load_iris(return_X_y=True)

In [ ]:
KM = KMeans(n_clusters=3, random_state=1).fit(X)

 fit encado el conjunto de datos, le aplico el ajuste.

In [ ]:
KM.cluster_centers_

In [ ]:
KM.inertia_

#Calidad de la partición. Coeficiente [**Silhouette**](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation).

EL coefiente de  **Silhouette** está definido para cada ejemplo y consta de dos mediadas:

- **a:** La media de la distancia entre un punto y **todos los puntos de su grupo**
- **b:** La media de la distancia entre un punto y **todos los puntos de su grupo más cercano** distinto al suyo 

el coeficiente para un punto simple se calcula como: $s=\frac{b-a}{max(a,b)}$

El coeficiente global se calcula como la media de los coeficientes de cada punto.

**Cuanto mayor es el valor del coefiente mejor es la agrupación**

relación que existe entre la similitud de los elementos con los elementos de su propia clase respecto a los de otras clases. Lo que se parecen a los de su propio grupo. 
La media es la distancia, por tanto debe ser pequeña. 

Coeficiente de agrupamiento:
Similitud entre los elementos y se diferencia del resto de elementos.

In [ ]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

Un uso normal del coeficiente de **Silhouette**

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.silhouette_score(X, labels, metric='euclidean')

La distancia se la indico. Podemos cambiar la distancia. 
La incercia va mejorando aumentando el número de cluster. 

# visualización

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
cols =  ['Class', 'Alcohol', 'MalicAcid', 'Ash', 'AlcalinityOfAsh', 'Magnesium', 'TotalPhenols', 
         'Flavanoids', 'NonflavanoidPhenols', 'Proanthocyanins', 'ColorIntensity', 
         'Hue', 'OD280/OD315', 'Proline']
data = pd.read_csv(url, names=cols)

In [ ]:
y=data['Class']
X=data.loc[:,'Alcohol':]

## Por piezas en dos dimensiones

In [ ]:
# three different scatter series so the class labels in the legend are distinct
plt.scatter(X[y==1]['Flavanoids'], X[y==1]['NonflavanoidPhenols'], label='Class 1', c='red')
plt.scatter(X[y==2]['Flavanoids'], X[y==2]['NonflavanoidPhenols'], label='Class 2', c='blue')
plt.scatter(X[y==3]['Flavanoids'], X[y==3]['NonflavanoidPhenols'], label='Class 3', c='lightgreen')

# Prettify the graph
plt.legend()
plt.xlabel('Flavanoids')
plt.ylabel('NonflavanoidPhenols')

# display
plt.show()

Asi estan distribuidos los vinos con respecto a la clase y flavanoids. 

## Reducción de la dimensionalidad

### Algoritmo **PCA**

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import MinMaxScaler
# creamos el objeto an abs_scaler
scaler = MinMaxScaler()
scaler.fit(X)
X_norm=scaler.transform(X)

In [ ]:
pca = sklearnPCA(n_components = 2) #2-dimensiones LDA
transformed = pd. DataFrame (pca.fit_transform(X_norm,y))
plt.scatter(transformed[y==1][0], transformed[y==1][1], label='Class 1', c='red')
plt.scatter(transformed[y==2][0], transformed[y==2][1], label='Class 2', c='blue')
plt.scatter(transformed[y==3][0], transformed[y==3][1], label='Class 3', c='lightgreen')

plt.legend()
plt.show()

Una componente principal a partir de dos combinaciones, no tiene un sentido como en el anterior. Lo que si es verdad, me sirve para mostrar que el custer que he hecho es correcto. 

### Algoritmo **LDA**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
# creamos el objeto an abs_scaler
scaler = MinMaxScaler()
scaler.fit(X)
X_norm=scaler.transform(X)

In [ ]:
lda = LDA(n_components=2) #2-dimensional LDA
lda_transformed = pd.DataFrame(lda.fit_transform(X_norm, y))

# Plot all three series
plt.scatter(lda_transformed[y==1][0], lda_transformed[y==1][1], label='Class 1', c='red')
plt.scatter(lda_transformed[y==2][0], lda_transformed[y==2][1], label='Class 2', c='blue')
plt.scatter(lda_transformed[y==3][0], lda_transformed[y==3][1], label='Class 3', c='lightgreen')

# Display legend and show plot
plt.legend(loc=3)
plt.show()

Valores de decisión